In [1]:
# !conda install sqlalchemy -y

import sqlalchemy

In [2]:
sqlalchemy.__version__

'1.4.7'

# Connecting to database

In [3]:
engine = sqlalchemy.create_engine('sqlite:///enterprise.db', echo=True)

# Declaring the mapping

In [4]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String

In [5]:
Base = declarative_base()

In [6]:
class User(Base):
    __tablename__ = 'users'   # required
    
    id = Column(Integer, primary_key=True)   # required
    name = Column(String(50))
    fullname = Column(String(50))
    age = Column(Integer)
    
    def __repr__(self):
        return "<User(name={}, fullname={}, age={})>".format(
        self.name, self.fullname, self.age)

# Creating the table on the database

In [7]:
Base.metadata.create_all(engine)

2023-05-07 15:59:46,011 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-07 15:59:46,014 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2023-05-07 15:59:46,015 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-07 15:59:46,016 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2023-05-07 15:59:46,017 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-07 15:59:46,018 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR(50), 
	fullname VARCHAR(50), 
	age INTEGER, 
	PRIMARY KEY (id)
)


2023-05-07 15:59:46,019 INFO sqlalchemy.engine.Engine [no key 0.00059s] ()
2023-05-07 15:59:46,212 INFO sqlalchemy.engine.Engine COMMIT


# Creating instances of the class

In [8]:
user = User(name='Enzo', fullname='Francisco Enzo', age=20)

In [9]:
user.name

'Enzo'

# Creating a session

In [10]:
from sqlalchemy.orm import sessionmaker

In [11]:
Session = sessionmaker(bind=engine)
Session

sessionmaker(class_='Session', bind=Engine(sqlite:///enterprise.db), autoflush=True, autocommit=False, expire_on_commit=True)

In [12]:
session = Session()

# Add objects

In [13]:
session.add(user)

In [14]:
session.commit()

2023-05-07 15:59:46,815 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-07 15:59:46,816 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, age) VALUES (?, ?, ?)
2023-05-07 15:59:46,817 INFO sqlalchemy.engine.Engine [generated in 0.00054s] ('Enzo', 'Francisco Enzo', 20)
2023-05-07 15:59:46,856 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
session.add_all([
    User(name='Valentina', fullname='Maria Valentina', age=23),
    User(name='Pedro', fullname='João Pedro', age=20)
])

In [16]:
# pending objects recently *added* to the Session
session.new

IdentitySet([<User(name=Valentina, fullname=Maria Valentina, age=23)>, <User(name=Pedro, fullname=João Pedro, age=20)>])

In [17]:
session.commit()

2023-05-07 15:59:47,298 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-07 15:59:47,299 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, age) VALUES (?, ?, ?)
2023-05-07 15:59:47,299 INFO sqlalchemy.engine.Engine [cached since 0.4832s ago] ('Valentina', 'Maria Valentina', 23)
2023-05-07 15:59:47,320 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, age) VALUES (?, ?, ?)
2023-05-07 15:59:47,321 INFO sqlalchemy.engine.Engine [cached since 0.5054s ago] ('Pedro', 'João Pedro', 20)
2023-05-07 15:59:47,323 INFO sqlalchemy.engine.Engine COMMIT


# Query objects 

In [18]:
query_user = session.query(User).filter_by(name='Enzo').first()

2023-05-07 15:59:47,481 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-07 15:59:47,483 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.age AS users_age 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2023-05-07 15:59:47,484 INFO sqlalchemy.engine.Engine [generated in 0.00058s] ('Enzo', 1, 0)


In [19]:
query_user

<User(name=Enzo, fullname=Francisco Enzo, age=20)>

In [20]:
query_user.id

1

In [21]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname, instance.age)

2023-05-07 15:59:47,772 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.age AS users_age 
FROM users ORDER BY users.id
2023-05-07 15:59:47,773 INFO sqlalchemy.engine.Engine [generated in 0.00106s] ()
Enzo Francisco Enzo 20
Valentina Maria Valentina 23
Pedro João Pedro 20


In [22]:
for info in session.query(User.name, User.age).order_by(User.id):
    print(info)

2023-05-07 15:59:47,873 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name, users.age AS users_age 
FROM users ORDER BY users.id
2023-05-07 15:59:47,874 INFO sqlalchemy.engine.Engine [generated in 0.00056s] ()
('Enzo', 20)
('Valentina', 23)
('Pedro', 20)


# Modify objects

In [23]:
user = session.query(User).filter_by(name='Enzo').first()
user

2023-05-07 15:59:48,049 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.age AS users_age 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2023-05-07 15:59:48,050 INFO sqlalchemy.engine.Engine [cached since 0.5668s ago] ('Enzo', 1, 0)


<User(name=Enzo, fullname=Francisco Enzo, age=20)>

In [24]:
user.fullname = "Francisco Enzo de Souza"

In [25]:
user

<User(name=Enzo, fullname=Francisco Enzo de Souza, age=20)>

In [26]:
# persistent objects which currently have *changes* detected
session.dirty

IdentitySet([<User(name=Enzo, fullname=Francisco Enzo de Souza, age=20)>])

In [27]:
session.commit()

2023-05-07 15:59:48,558 INFO sqlalchemy.engine.Engine UPDATE users SET fullname=? WHERE users.id = ?
2023-05-07 15:59:48,560 INFO sqlalchemy.engine.Engine [generated in 0.00161s] ('Francisco Enzo de Souza', 1)
2023-05-07 15:59:48,562 INFO sqlalchemy.engine.Engine COMMIT


# Delete objects

In [28]:
user = session.query(User).filter_by(name='Valentina').first()

2023-05-07 15:59:48,744 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-07 15:59:48,745 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.age AS users_age 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2023-05-07 15:59:48,745 INFO sqlalchemy.engine.Engine [cached since 1.262s ago] ('Valentina', 1, 0)


In [29]:
user

<User(name=Valentina, fullname=Maria Valentina, age=23)>

In [30]:
session.delete(user)

In [31]:
session.new

IdentitySet([])

In [32]:
session.dirty

IdentitySet([])

In [33]:
# persistent objects that have been marked as *deleted* via session.delete(obj)

session.deleted

IdentitySet([<User(name=Valentina, fullname=Maria Valentina, age=23)>])

In [34]:
session.commit()

2023-05-07 15:59:49,375 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2023-05-07 15:59:49,376 INFO sqlalchemy.engine.Engine [generated in 0.00098s] (2,)
2023-05-07 15:59:49,420 INFO sqlalchemy.engine.Engine COMMIT


In [35]:
session.query(User).filter_by(name='Valentina').count()

2023-05-07 15:59:49,588 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-07 15:59:49,590 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.age AS users_age 
FROM users 
WHERE users.name = ?) AS anon_1
2023-05-07 15:59:49,591 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ('Valentina',)


0